<a href="https://colab.research.google.com/github/kosirobwada/SIGNATE/blob/main/beginner37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SIGNATE/beginner37/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SIGNATE/beginner37/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SIGNATE/beginner37/sample_submit.csv')

In [ ]:
print('train:', train.isnull().sum())
print('test:', test.isnull().sum())

train: id              0
mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64
test: id              0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64


In [ ]:
# ?の個数を確認する
print(train[train['horsepower'] == '?'])

Empty DataFrame
Columns: [id, mpg, cylinders, displacement, horsepower, weight, acceleration, model year, origin, car name]
Index: []


In [ ]:
import numpy as np

# Replace '?' with NaN for the 'horsepower' column
train['horsepower'] = train['horsepower'].replace('?', np.nan)

# Convert 'horsepower' column to float
train['horsepower'] = train['horsepower'].astype(float)

# Fill NaN values in 'horsepower' with its mean
train['horsepower'] = train['horsepower'].fillna(train['horsepower'].mean())


In [ ]:
import numpy as np

# Replace '?' with NaN for the 'horsepower' column
test['horsepower'] = test['horsepower'].replace('?', np.nan)

# Convert 'horsepower' column to float
test['horsepower'] = test['horsepower'].astype(float)

# Fill NaN values in 'horsepower' with its mean
test['horsepower'] = test['horsepower'].fillna(train['horsepower'].mean())


In [ ]:
y = train['mpg']

X_train = train.drop(['id', 'mpg', 'car name'], axis=1)
X_test = test.drop(['id', 'car name'], axis=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# 評価手法
from sklearn.metrics import r2_score

In [ ]:
# 学習モデル
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# pipeline setting
pipelines = {
     'ols': Pipeline([('scl',StandardScaler()),
                      ('est',LinearRegression())]),

     'ridge':Pipeline([('scl',StandardScaler()),
                       ('est',Ridge(random_state=0))]),

     'tree': Pipeline([('scl',StandardScaler()),
                     ('est',DecisionTreeRegressor(random_state=0))]),

     'rf': Pipeline([('scl',StandardScaler()),
                     ('est',RandomForestRegressor(random_state=0))]),

     'gbr1': Pipeline([('scl',StandardScaler()),
                      ('est',GradientBoostingRegressor(random_state=0))]),

     'gbr2': Pipeline([('scl',StandardScaler()),
                      ('est',GradientBoostingRegressor(n_estimators=250,
                                                       random_state=0))])
}

In [ ]:
#pipelineを作成するために一旦作成
sample_pred = sample.iloc[:, 1]

In [ ]:
# X_testとsample_predのインデックスが完全に一致するか確認
if X_test.index.equals(sample_pred.index):
    print("X_testとsample_predのインデックスは一致しています。")
else:
    print("X_testとsample_predのインデックスは一致していません。")

    # 一致していない場合、差異を表示
    diff_indices_X_test = X_test.index.difference(sample_pred.index)
    diff_indices_sample_pred = sample_pred.index.difference(X_test.index)

    if not diff_indices_X_test.empty:
        print("X_testに存在して、sample_predに存在しないインデックス:", diff_indices_X_test)
    if not diff_indices_sample_pred.empty:
        print("sample_predに存在して、X_testに存在しないインデックス:", diff_indices_sample_pred)


X_testとsample_predのインデックスは一致していません。
X_testに存在して、sample_predに存在しないインデックス: RangeIndex(start=499, stop=500, step=1)


In [ ]:
X_test = X_test.drop(499)


In [ ]:
# build and evaluate
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y)
    scores[(pipe_name,'train')] = r2_score(y, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = r2_score(sample_pred, pipeline.predict(X_test))

print(pd.Series(scores).unstack())

       test     train
gbr1    0.0  0.933248
gbr2    0.0  0.974950
ols     0.0  0.649573
rf      0.0  0.976893
ridge   0.0  0.649570
tree    0.0  1.000000


In [ ]:
model = GradientBoostingRegressor(
    min_samples_split = 5,
    min_samples_leaf = 52,
    max_depth = 6,
    max_features = 'sqrt',
    subsample = 0.8)

model.fit(X_train, y)

GradientBoostingRegressor(max_depth=6, max_features='sqrt', min_samples_leaf=52,
                          min_samples_split=5, subsample=0.8)

In [ ]:
# 予測
pred = model.predict(X_test)
print(pred)

[35.74555868 28.57569476 26.19096296 31.77465053 29.06940641 36.43868031
 28.65923506 25.27584602 34.19886812 20.09081021 33.75798458 34.48068505
 20.86691958 25.884356   19.29171595 30.86220357 17.71751172 18.45208392
 33.78350386 35.21021863 32.54522523 34.30035099 22.44674281 21.37787861
 36.7307988  21.5016383  21.99098691 17.71751172 20.89876461 22.55915824
 36.50386767 24.84514091 18.52673718 25.20752613 21.65295155 36.66785153
 26.48394983 23.9347838  30.10022033 31.74154137 23.75143868 28.3137494
 32.75486342 34.28801156 30.258882   20.49939231 17.78628004 20.17141947
 36.56934462 27.64859714 17.23711361 32.0632843  23.83507441 35.23212292
 20.17141947 27.18813219 30.69851321 34.87354564 29.09648256 29.16953445
 31.26355146 17.94349184 24.92104713 26.65016853 21.78830383 32.27363388
 31.85050107 23.64712984 27.89560883 22.60829269 27.75584121 34.93254957
 36.16512212 20.84320398 32.35844212 17.71751172 24.27978706 19.72107044
 36.60534728 35.74555868 18.52673718 26.130717   34.

In [ ]:
sample[1] = pred
sample.to_csv('GradientBoostingRegressor.csv', index=None, header=None)

In [ ]:
param = {'n_estimators': list(range(20, 101, 10)),
             'learning_rate': list(np.arange(0.05, 0.20, 0.01))}


In [ ]:
# scoringを neg_mean_squared_error(負のMSE)とする
gsearch1 = GridSearchCV(estimator = model,
                        param_grid = param,
                        cv = 5,
                        n_jobs=4,
                        scoring = 'neg_mean_squared_error')


#fitさせる
gsearch1.fit(X_train, y)

GridSearchCV(cv=5,
             estimator=GradientBoostingRegressor(max_depth=6,
                                                 max_features='sqrt',
                                                 min_samples_leaf=52,
                                                 min_samples_split=5,
                                                 subsample=0.8),
             n_jobs=4,
             param_grid={'learning_rate': [0.05, 0.060000000000000005, 0.07,
                                           0.08000000000000002,
                                           0.09000000000000001, 0.1,
                                           0.11000000000000001,
                                           0.12000000000000001, 0.13, 0.14,
                                           0.15000000000000002,
                                           0.16000000000000003,
                                           0.17000000000000004,
                                           0.18000000000000005, 0.19, 0.2],
                         'n_estimators': [20, 30, 40, 50, 60, 70, 80, 90, 100]},
             scoring='neg_mean_squared_error')

In [ ]:
# test精度の平均が最も高かった組み合わせを出力
print(gsearch1.best_params_)

{'learning_rate': 0.18000000000000005, 'n_estimators': 100}


In [ ]:
# 予測
pred_gsearch = gsearch1.predict(X_test)

sample[1] = pred_gsearch
sample.to_csv('GBR_pred_gsearch.csv', index=None, header=None)

In [ ]:
params = {'n_estimators'  : [10, 100, 500, 2000], 'n_jobs': [-1]}

# モデルにインスタンス生成
mod = RandomForestRegressor()

# ハイパーパラメータ探索
cv = GridSearchCV(mod, params, cv = 10, scoring= 'neg_mean_squared_error', n_jobs =1)
cv.fit(X_train, y)

# 予測
pred_cv = cv.predict(X_test)

sample[1] = pred_cv
sample.to_csv('RFR_gridserch.csv', index=None, header=None)

In [ ]:
import pandas as pd

# CSVファイルの読み込み
df = pd.read_csv('/content/RFR_gridserch.csv')
# 左から2行目を削除
df = df.drop(1)

# 変更をCSVファイルに保存
df.to_csv('submit.csv', index=False)